### import packages
---

In [1]:
import pandas as pd
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
# set up display area to show dataframe in jupyter qtconsole
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 3)

import xlrd
import xlsxwriter

import math
import time
from time import sleep
import schedule

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib import gridspec

import mplfinance as mpf
import yfinance as yf
from qtrade import Questrade

from datetime import date
from datetime import datetime
from datetime import timedelta

In [2]:
def getMaxPeriod(interval="1m"):
    #1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
    if interval == "1m":
        Maxperiod = "7d"
        
    elif interval == "2m":
        Maxperiod = "60d"
        
    elif interval == "5m":
        Maxperiod = "60d"
        
    elif interval == "15m":
        Maxperiod = "60d"
        
    elif interval == "30m":
        Maxperiod = "60d"
        
    elif interval == "60m":
        Maxperiod = "60d"
        
    elif interval == "90m":
        Maxperiod = "60d"
         
    elif interval == "1h":
        Maxperiod = "60d"       
                
    elif interval == "1d":
        Maxperiod = "60d"
                
    elif interval == "5d":
        Maxperiod = "60d"

    elif interval == "1wk":
        Maxperiod = "60d"

    elif interval == "1mo":
        Maxperiod = "60d"

    elif interval == "3mo":
        Maxperiod = "60d"
    else:
        print("Please enter appropraite Chart Interval")
        print("Program has stopped")
    return Maxperiod



In [3]:
def getChartLoop(Maxperiod="7d", ChartPeriodYrs=2):
    if Maxperiod == "7d":
        MaxPeriodDT = timedelta(days=7)
    elif Maxperiod == "60d":
        MaxPeriodDT = timedelta(days=60)
    
    ChartPeriodDays = ChartPeriodYrs * 365
    ChartPeriodDT = timedelta(days=ChartPeriodDays)    
    
    looper = int(ChartPeriodDT / MaxPeriodDT)
    print(looper)
    return (looper,MaxPeriodDT)
   


In [4]:
def getDatePairLst(LoopQty, MaxPeriodDT):
    # Current date time in local system
    #enddate = datetime.date(datetime.now())
    today = date.today()
    dates = []
    dates.append(today)
    for timeCount in range(1, LoopQty + 1 ):
        backDays = timeCount * MaxPeriodDT
        pastDate = today - backDays
        pastDateStr = pastDate.strftime("%Y-%m-%d")
        dates.append(pastDateStr)
    
    dateslen = (len(dates))
    datelst = []
    for nos in range(dateslen - 1):
        if nos != dateslen - 1:
            print(nos,dateslen)
            datepair = (dates[nos], dates[nos + 1])
            #print("{} and {}".format(datepair, nos))
            datelst.append(datepair)
        
    
        
        #past_day = today.day
            #if past_day == 31:
                #past_day = 29
            #past_month = (today.month - month) % 12
            #if past_month == 0:
                #past_month = 12
            #past_year = today.year - ((today.month + month) // 12)
        
            #pastdate_str = "{} {} {}".format(past_year, past_month, past_day)
            #pastdate = datetime.strptime(pastdate_str, "%Y %m %d")
            #dates.append(pastdate)
            #print("month: {}, date: {}".format(month, pastdate))           
        
        #print(len(dates))   

In [5]:
def getAllTickers(DatePairlst):
    Tickers = ['brtx','gree']
    for ticker in Tickers:
        getTickersDf(ticker, DatePairlst)

            

In [6]:
def getTickersDf(tickers, DatePairlst):
    main_lst = []
    for DatePair in DatePairlst:
        start,end = DatePair
        df = df_single_get(ticker, interval, start, end)
        main_lst.append(df)
    large_df = pd.concat(main_lst, ignore_index=True)
    return large_df
    

###    function to get single ticker info from yfinance

In [7]:
def df_single_get(ticker, minterval, mstart , mend):  # function used to get single dataframe for a single ticker
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    # download pre/post regular market hours data
    # (optional, default is False) prepost=False
    df = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        ticker,

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        #period="60d",
        #start="2021-03-01",
        #end="2021-11-13",
        start = mstart,
        end = mstart,

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        #interval="30m",
        interval = minterval,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        # group_by='ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        # auto_adjust=True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost=True,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads=True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        # proxy=None
    )

    df.insert(0, 'Ticker', ticker, True)
    return df

In [8]:
interval="2m"
Maxperiod = getMaxPeriod(interval)    
ChartPeriodYrs = 2    
LoopQty, MaxPeriodDT = getChartLoop(Maxperiod, ChartPeriodYrs)
DatePairlst = getDatePairLst(LoopQty, MaxPeriodDT)
getAllTickers(DatePairlst)

12
0 13
1 13
2 13
3 13
4 13
5 13
6 13
7 13
8 13
9 13
10 13
11 13


TypeError: 'NoneType' object is not iterable